In [6]:
import pandas as pd
import requests
import time
import numpy as np



# country_list = [
#     'Antigua and Barbuda', 'Bahamas', 'Barbados', 'Belize', 'Canada',
#     'Costa Rica', 'Cuba', 'Dominica', 'Dominican Republic', 'El Salvador',
#     'Grenada', 'Guatemala', 'Haiti', 'Honduras', 'Jamaica', 'Mexico',
#     'Nicaragua', 'Panama', 'Saint Kitts and Nevis', 'Saint Lucia',
#     'Saint Vincent and the Grenadines', 'Trinidad and Tobago', 'United States'
# ]

country_list = ['storm surge', 'hurricane']
api_key = 'XBbGPcpPXB5CH0aLSu6BO5fww06zAusz6dFGXqj0'



headers = {
    'x-api-key': api_key
}

max_limit = 100  # The maximum limit per request
max_offset = 1000  # The maximum offset

for this_year in range(2017, 2025):
    for this_country in country_list:
        paper_id_list = []
        url_list = []
        title_list = []
        abstract_list = []
        authors_list = []
        year_list = []
        citationCount_list = []
        keywords = ['water level', 'satellite', 'predict', this_country]

        query = '+'.join(keywords)
        this_date = str(this_year)+'-'+f"{this_month:02}"
        results_url = f"https://api.semanticscholar.org/graph/v1/paper/search?query={query}&publicationDateOrYear={this_year}&fields=title,url,abstract,year,authors,citationCount"
        print(results_url)
        # Fetch the total number of results
        response = requests.get(results_url, headers=headers)
        if response.status_code != 200:
            print(f"Failed to fetch data for {query} in {this_year}. Status code: {response.status_code}")
            continue
        data = response.json()
        print(data.get('total', 0))

        if data.get('total', 0) > 1000:

            for this_month in range(1, 13):
                    
                keywords = ['water level', 'satellite', 'time series','computer vision', this_country]
                query = '+'.join(keywords)
                this_date = str(this_year)+'-'+f"{this_month:02}"
                results_url = f"https://api.semanticscholar.org/graph/v1/paper/search?query={query}&publicationDateOrYear={this_date}&fields=title,url,abstract,year,authors,citationCount"
                print(results_url)
                # Fetch the total number of results
                response = requests.get(results_url, headers=headers)
                if response.status_code != 200:
                    print(f"Failed to fetch data for {query} in {this_year}. Status code: {response.status_code}")
                    continue
                data = response.json()
                print(data.get('total', 0))
                total_number = min(data.get('total', 0), max_offset)  # Ensure we don't exceed 1000
                print(total_number)
                count_offset = 0
                total_data = []
                while count_offset < total_number:
                    limit = min(max_limit, total_number - count_offset)  # Adjust limit if nearing total_number
                    this_results_url = f"{results_url}&offset={count_offset}&limit={limit}"
                    response = requests.get(this_results_url, headers=headers)
                    if response.status_code != 200:
                        print(f"Failed to fetch data for offset {count_offset}. Status code: {response.status_code}")
                        break

                    data = response.json()
                    papers = data.get('data', [])
                    if not papers:
                        print("No data found at this offset.")
                        break

                    total_data.extend(papers)
                    count_offset += limit
                    time.sleep(1)  # To respect API rate limits

                for this_paper_id in range(len(total_data)):
                            this_paper = total_data[this_paper_id]
                            abstract = this_paper.get('abstract')
                            if abstract and ('satellite' in abstract):
                                print(this_paper_id)
                                paper_id_list.append(this_paper.get('paperId', 'N/A'))
                                url_list.append(this_paper.get('url', 'N/A'))
                                title_list.append(this_paper.get('title', 'N/A'))
                                abstract_list.append(this_paper.get('abstract', 'N/A'))
                                authors_list.append(', '.join([author['name'] for author in this_paper.get('authors', [])]))
                                year_list.append(this_paper.get('year', 'N/A'))
                                citationCount_list.append(this_paper.get('citationCount', 0))
        else:
            count_offset = 0
            total_data = []
            while count_offset < total_number:
                limit = min(max_limit, total_number - count_offset)  # Adjust limit if nearing total_number
                this_results_url = f"{results_url}&offset={count_offset}&limit={limit}"
                response = requests.get(this_results_url, headers=headers)
                if response.status_code != 200:
                    print(f"Failed to fetch data for offset {count_offset}. Status code: {response.status_code}")
                    break

                data = response.json()
                papers = data.get('data', [])
                if not papers:
                    print("No data found at this offset.")
                    break

                total_data.extend(papers)
                count_offset += limit
                time.sleep(1)  # To respect API rate limits

            for this_paper_id in range(len(total_data)):
                        this_paper = total_data[this_paper_id]
                        abstract = this_paper.get('abstract')
                        if abstract and ('satellite' in abstract):
                            print(this_paper_id)
                            paper_id_list.append(this_paper.get('paperId', 'N/A'))
                            url_list.append(this_paper.get('url', 'N/A'))
                            title_list.append(this_paper.get('title', 'N/A'))
                            abstract_list.append(this_paper.get('abstract', 'N/A'))
                            authors_list.append(', '.join([author['name'] for author in this_paper.get('authors', [])]))
                            year_list.append(this_paper.get('year', 'N/A'))
                            citationCount_list.append(this_paper.get('citationCount', 0))
        if len(title_list) == 0:
            continue
        paper_repos_dict = {
            'title': title_list,
            'abstract': abstract_list,
            'year': year_list,
            'citationCount': citationCount_list,
            'url': url_list,
            'paperId': paper_id_list,
            'authors': authors_list
        }

        this_paper_sum = pd.DataFrame(paper_repos_dict)
        this_paper_sum.to_csv(f'/Users/liting/Documents/GitHub/google-scholar-search/storm_surge_search/{"_".join(keywords)}_{this_year}.csv', sep=',', index=False, header=True)


https://api.semanticscholar.org/graph/v1/paper/search?query=water level+satellite+predict+storm surge&publicationDateOrYear=2017&fields=title,url,abstract,year,authors,citationCount
214
3
https://api.semanticscholar.org/graph/v1/paper/search?query=water level+satellite+predict+hurricane&publicationDateOrYear=2017&fields=title,url,abstract,year,authors,citationCount
58
2
3
4
5
6
https://api.semanticscholar.org/graph/v1/paper/search?query=water level+satellite+predict+storm surge&publicationDateOrYear=2018&fields=title,url,abstract,year,authors,citationCount
266
https://api.semanticscholar.org/graph/v1/paper/search?query=water level+satellite+predict+hurricane&publicationDateOrYear=2018&fields=title,url,abstract,year,authors,citationCount
59
3
4
5
6
https://api.semanticscholar.org/graph/v1/paper/search?query=water level+satellite+predict+storm surge&publicationDateOrYear=2019&fields=title,url,abstract,year,authors,citationCount
276
https://api.semanticscholar.org/graph/v1/paper/search?qu

In [7]:
import os
import pandas as pd

def read_all_csv_files_in_directory(directory_path):
    # List all files in the directory
    files = os.listdir(directory_path)
    # Filter out only CSV files
    csv_files = [file for file in files if file.endswith('.csv')]
    
    # List to hold dataframes
    dataframes = []
    
    # Iterate over CSV files and read them into a list of dataframes
    for csv_file in csv_files:
        file_path = os.path.join(directory_path, csv_file)
        df = pd.read_csv(file_path)
        dataframes.append(df)
    
    # Optionally, concatenate all dataframes into a single dataframe
    combined_df = pd.concat(dataframes, ignore_index=True)
    
    return combined_df, dataframes

# Example usage
directory_path = '/Users/liting/Documents/GitHub/google-scholar-search/storm_search'
combined_df, dataframes = read_all_csv_files_in_directory(directory_path)
combined_df=combined_df.drop_duplicates()
combined_df.to_csv(f'/Users/liting/Documents/GitHub/google-scholar-search/storm_surge_search/combined_df.csv', sep=',', index=False, header=True)